- нужено описать и реализовать распределенный генератор случайных чисел
основная статья — https://ieeexplore.ieee.org/abstract/document/7520905
  
#### Requirements  
 - Unpredictability. (Nearly) Impossible to predict next rn given the knowledge of all the past rn-s.  
    Note. Most of algorithms consists of two steps: entropy collection and entropy revealing. Unpredictability means impossibility to read result or make wanted number to be a result before entropy collection step.
 - Finality. All honest participants must generate the same rn.
 - Liveness. A rn will eventually be generated.
 - Unbiasedness. Distribution of rn cannot be influenced by a (small?) group of participants. In practice, it means that the generation ceremony cannot be influenced by selective participation of malicious participants.
 - Sufficiency. There are “enough” rn, i.e., rn-s are available at request.  

#### BFT RNG (in collaboration with Mikhail Krasnoselsky)

 - Statement 1. Let X1 be a uniform on {0, 1, …, N-1} random variable, which is independent of arbitrary measurable random variable X2. Then X1+ X2 mod N is the uniform on {0, 1, …, N-1} random variable.
 - Statement 2. Let X11 , …, X1M be a i.i.d. uniform on {0, 1, …, N-1} random variables, which is independent of arbitrary measurable random variable X21, …, X2M. Then X11+ X21 mod N, …, X1M+ X2M mod N are independent uniform on {0, 1, …, N-1}.  

#### Proposed algorithm:
 - secret sharing without a dealer based on polynomials and homomorphic encryption (https://ieeexplore.ieee.org/abstract/document/7520905).
 - Resulting RN – the value of a secret polynomial at a given point (for example, 0).

#### Properties
 - Each step results in unpredictable RN or revealing at least one faulty process. A faulty processes could only rip off during the “preparation step” without ability to influence on unpredictability.
 - Limitation: we need to know the list of participants as always in BFT.   
 - Other limitations:  
    Liveness: h >= d + 1, where h is the number of honest participants  
    Unpredictability: h >= 2 and f <= d + 1, where f is the number of faulty participants.  
    Classic result about BFT RNG: https://link.springer.com/chapter/10.1007/11945529_20 (разбор есть на нашей wiki).  

#### Related papers (RNG usage for PoS):  
    https://fc16.ifca.ai/bitcoin/papers/BGM16.pdf  
    https://link.springer.com/chapter/10.1007/978-3-319-63688-7_12  


- То есть сейчас, в качестве подготовительного шага, надо сделать демо генерации случайного полинома как суммы случайных полиномов в системе из n участников, где n известно. Никакого блокчейна, просто обмен сообщениями (можно даже сеть не моделировать — просто набор функций для имитации). Язык — на ваше усмотрение (Python устроит).

In [1]:
# Количество участников:

n = 10
degree = n - 1

- В качестве homomorphic one-way function that satisfies $E(x + y) = E(x)*E(y)$ пока возьмём $E(x) = g^x$ над $Z_p^*$

In [2]:
import math
import random

class EClass:
    def __init__(self, m, g = 13, n = 101):
        self.m = m
        self.g = g
        self.n = n
        
    def __mul__(self, other):
        return EClass(self.m + other.m)
    
    def __imul__(self, other):
        return self * other
    
    def calc(self):
        return self.g**(self.m % self.n)

In [3]:
A = EClass(10)
B = EClass(20)

print(f"E(x)*E(y) = {(A*B).calc()}")
print(f"E(x+y)    = {EClass(10 + 20).calc()}")

E(x)*E(y) = 2619995643649944960380551432833049
E(x+y)    = 2619995643649944960380551432833049


#### Для работы с полиномами сделаем класс, который поддерживает:
- сложение полиномов
- генерацию случайного полинома по заданной степени и значению в точке

In [4]:
class Polinomial:
    def __init__(self, coef = [0]):
        if type(coef) == int:
            coef = [coef]
        self.self_update(coef)

    def self_update(self, coef = []):
        new_coef = [0]
        for i, c in enumerate(coef[::-1]):
            if c:
                new_coef = coef[:len(coef)-i]
                break
        self.coef = new_coef
        self.degree = len(new_coef) - 1
                
    def generate_polynomial(self, degree, a, b):
        """chooses coefficients for a polynomial of the given degree, such that f(a) == b"""

        #to fit only one data point, we can choose arbitrary values for every coefficient except one, which we initially set to zero.
        coefficients = [0] + [random.randint(1, 10) for _ in range(degree)]

        #now calculate f(a). This will probably not be equal to b, initially.
        y = sum(coefficient * a**n for n, coefficient in enumerate(coefficients))

        #setting the final coefficient to their difference will cause f(a) to equal b.
        coefficients[0] = b - y

        self.self_update(coefficients)
        
    def __getitem__(self, x):
        res = 0
        for i in range(self.degree + 1):
            res += self.coef[i] * x ** i
        return res
    
    def __add__(self, other):
        A, B = self.coef, other.coef
        if self.degree > other.degree:
            A, B = B, A
        A = A + [0] * (other.degree - self.degree)
        return Polinomial([A[i] + B[i] for i in range(len(A))])
        
    
    def __str__(self):
        poli = ""
        coef = list(map(str,self.coef[::-1]))
        for i in range(self.degree + 1):
            poli += coef[i]
            if i != self.degree:
                poli += ("x^" + str(self.degree - i) + " + ")
        return poli

Примеры случайных полиномов:

In [5]:
p = Polinomial()
p.generate_polynomial(degree = 9, a = 0, b = 10)
print(p)
p.generate_polynomial(degree = 9, a = 0, b = 10)
print(p)

8x^9 + 7x^8 + 4x^7 + 5x^6 + 4x^5 + 4x^4 + 2x^3 + 9x^2 + 2x^1 + 10
4x^9 + 2x^8 + 10x^7 + 5x^6 + 5x^5 + 6x^4 + 4x^3 + 9x^2 + 10x^1 + 10


#### Для исключения ошибки вычислений вещественных чисел при интерполяции введём класс простых дробей с поддержкой:
- сложения
- умножения
- приведения к десятичной дроби

In [6]:
class Fraction:
    def __init__(self, n, d = None):
        self.n = n
        if not d:
            d = 1
        self.d = d
        
    def __add__(self, other):
        if type(other) == int:
            other = Fraction(other)
        return Fraction(self.n * other.d + self.d * other.n, self.d * other.d)
    
    def __iadd__(self, other):
        return self + other
    
    def __mul__(self, other):
        if type(other) == int:
            other = Fraction(other)
        return Fraction(self.n * other.n, self.d * other.d)
    
    def __imul__(self, other):
        return self * other
    
    def to_decimal(self):
        return self.n / self.d
    
    def __str__(self):
        return f"{self.n} / {self.d}"  

#### Этап генерации секрета:
- В secret_I лежат оригинальные I
- В public_I лежат E(I) для публичной передачи

In [7]:
def secret_gen_part():
    s_I = random.randint(100000, 1000000)
    p_I = EClass(s_I)
    return s_I, p_I

def secret_gen(n):
    secret_I = [None] * n
    public_I = [None] * n
    fs = [None] * n
    for i in range(n):
        s_I, p_I = secret_gen_part()

        secret_I[i] = s_I
        public_I[i] = p_I

        fs[i] = Polinomial()
        fs[i].generate_polynomial(degree, 0, s_I)
    return secret_I, public_I, fs

In [8]:
secret_I, public_I, fs = secret_gen(n)

#### Этап расшаривания секрета:
- в x лежат публичные x_i для каждого участника
- в res_y части секрета каждого участника

In [9]:
def secret_sharing_part_init(I, f1):
    r = random.randint(100, 1000)
    x = random.randint(100, 1000)
    return r, x, f1, f1[x] + r

def secret_sharing_part(x, fi, y):
    return fi[x] + y

def secret_sharing(n, secret_I, fs):
    r = [None] * n 
    x = [None] * n 
    y = [[None] * n for i in range(n)]

    for i in range(n):
        r[i], x[i], fs[i], y[i][(i+1) % n] = secret_sharing_part_init(secret_I[i], fs[i])
        for j in range(1,n):
             y[i][(j+i+1) % n] = secret_sharing_part(x[i], fs[(j+i) % n], y[i][(j+i) % n])

    res_y = [y[i][i] - r[i] for i in range(n)]
    return x, res_y

In [10]:
x, y = secret_sharing(n, secret_I, fs)

#### Этап реконструкции секрета
Считаем, что n участников передали свои доли, тогда восстанавливаем полином степени n-1 (не собираем его в явном виде, т.к. нам нужно значение лишь в одной точке)

In [11]:
def interpolation(x, y, x0 = 0):
    n = len(x)
    res = Fraction(0,1)
    for i in range(n):        
        li = Fraction(1,1)
        for j in range(n):
            if j == i:
                continue
            li = li * Fraction((x0 - x[j]), (x[i] - x[j]))
        res = res + Fraction(y[i], 1)*li
    return res.to_decimal()

In [12]:
interpolation(x, y, x0 = 0)

5435100.0

Подсчитываем $\prod_{i = 1}^n E(I_i)$ и сравниваем с $E(S')$

In [13]:
E_mul = public_I[0]
for p_I in public_I[1:]:
    E_mul *= p_I

In [14]:
E_mul.calc() == EClass(int(interpolation(x, y))).calc()

True

Сходится!